In [1]:
import os

#initialize the params
target="\\Cali\\4K\\CAM2"

#change directory for saving images
if os.getcwd().endswith(target)==False:
    PATH=os.getcwd() + target
    os.chdir(PATH)

print(os.getcwd())

c:\Users\User\Desktop\Dev\tabletennis_robot\Cali\4K\CAM2


In [9]:
import cv2

camera_num=0
cap=cv2.VideoCapture(camera_num)

#change camera frame to 1280 X 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

#print changed frame
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print('changed resolution width {} height {}'.format(width, height))

changed resolution width 1280.0 height 720.0


In [11]:
#capture the checkerboard
import cv2
import os

#initialize the params
pic_num=0

camera_num=0
cap=cv2.VideoCapture(camera_num)

#change camera frame to 1280 X 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

if cap.isOpened():
    #print the information of current camera
    print("frame type: ", cap.get(cv2.CAP_PROP_FRAME_TYPE))
    print("frame height: ", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("frame width: ", cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    #capture the checkerboard images
    while True:
        ret, img=cap.read()
        if ret:
            cv2.imshow('camera', img)
            # command=input("Waiting for yout keyboard press: c for capture, q for quit")
            command=cv2.waitKey(1)
            if command==ord("c"):
                    cv2.imwrite(f"checkerboard{pic_num}_cam{camera_num}.jpg", img)
                    pic_num+=1
            elif command==ord("q"):
                break
        else:
            print("no frame!")
            break
    cap.release()
else:
    print("cannot open camera")
cap.release()
cv2.destroyAllWindows()

frame type:  -1.0
frame height:  720.0
frame width:  1280.0


In [2]:
#with checkerboard images, do calibration
import cv2
import numpy as np
import os
import glob


#define the board's dimension
CHECKERBOARD=(4, 5)
criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#generate vector of 3D points
objpoints=[]

#generate vector of 2D points
imgpoints=[]

#define world matrix
objp=np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2]=np.mgrid[0: CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape=None

#extract the path of checkerboard images
# target='./Cali/4K/CAM1/*.jpg'

images=glob.glob(os.getcwd()+"/*.jpg")
print(images)


for fname in images:
    img=cv2.imread(fname)

    #invert into gray scale
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #find corners of the checkerboard
    ret, corners=cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    print("cv2 corner detection result: ", ret)
    
    #if the desired number of corners is found
    if ret==True:
        objpoints.append(objp)
        
        #fine tunning the matrix based on given 2D points
        corners2=cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria) 
        imgpoints.append(corners2)
        # print("corners2: ", corners2)

        #illustrate the corners
        img=cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

    # elif ret==False:
    #     os.remove(fname)
        
    cv2.imshow('cali result', img)
    cv2.waitKey(0)



cv2.destroyAllWindows()
h, w=img.shape[:2]
print("h, w: ", h, w)

#check the results
print("objpoints: ", objpoints)

#with the true 3D points and given 2D points, do calibration
ret, mtx, dist, rvecs, tvecs=cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


['c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard0_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard104_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard110_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard111_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard124_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard126_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard127_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard128_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard139_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard145_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\

In [3]:
print("camera matrix: \n")
print(mtx)

camera matrix: 

[[509.50837051   0.         313.14423307]
 [  0.         507.64569231 244.83368204]
 [  0.           0.           1.        ]]


In [4]:
print("dist: \n")
print(dist)

dist: 

[[ 2.51864040e-01 -7.95519959e-01 -3.44454508e-04 -2.86255244e-03
   7.53547597e-01]]


In [5]:
print("rvecs: \n")
print(rvecs)

rvecs: 

(array([[-0.64034181],
       [-0.67301361],
       [-1.41130224]]), array([[-0.5476795 ],
       [-0.59290272],
       [-1.48819607]]), array([[-0.43735082],
       [-0.4085592 ],
       [-1.47029182]]), array([[-0.43464924],
       [-0.33350747],
       [-1.46994211]]), array([[-0.48267752],
       [-0.18178809],
       [-1.47257621]]), array([[-0.41564664],
       [-0.21060959],
       [-1.50130263]]), array([[-0.28054358],
       [-0.30471856],
       [-1.51459004]]), array([[-0.22096641],
       [-0.37203153],
       [-1.51807713]]), array([[-0.07456735],
       [-0.89445071],
       [-1.50338098]]), array([[-0.10509607],
       [-0.47959315],
       [-1.48846068]]), array([[-0.071004  ],
       [-0.42714833],
       [-1.51513389]]), array([[-0.49324185],
       [-0.38445542],
       [-2.35281482]]), array([[-0.70765861],
       [-0.5433967 ],
       [-2.26804965]]), array([[-0.77452021],
       [-0.51413261],
       [-2.23058914]]), array([[-0.8547297 ],
       [-0.39790

In [6]:
print("tvecs: \n")
print(tvecs)

tvecs: 

(array([[-3.43547266],
       [ 2.64804032],
       [10.94307363]]), array([[ 0.80047168],
       [ 2.52177376],
       [12.01772787]]), array([[ 0.31107391],
       [ 1.07251192],
       [13.19390663]]), array([[ 0.77116904],
       [ 1.45997368],
       [13.47968046]]), array([[ 0.15518486],
       [ 1.77723529],
       [12.5795111 ]]), array([[-1.01764824],
       [ 1.9602465 ],
       [12.82218228]]), array([[-1.88375943],
       [ 2.05644384],
       [12.47439588]]), array([[-2.46579649],
       [ 2.16845443],
       [12.21197932]]), array([[-2.80443841],
       [ 1.35898109],
       [11.0413415 ]]), array([[-2.30013864],
       [ 1.0589889 ],
       [11.24272919]]), array([[-2.16690423],
       [ 1.10780424],
       [11.38685064]]), array([[ 1.58602353],
       [ 3.47314621],
       [13.70514727]]), array([[ 1.73327607],
       [ 3.2057453 ],
       [14.07180257]]), array([[ 2.28801966],
       [ 3.07577627],
       [14.78556413]]), array([[ 3.38582236],
       [ 3.00337

In [7]:
#human error: current directory forgetting
print(os.getcwd())
target='./Cali/4K/CAM2'
images=glob.glob(target)
print(images)
for fname in images:
    print(fname)

c:\Users\User\Desktop\Dev\tabletennis_robot\Cali\4K\CAM2
[]
